In [ ]:
from flask import Flask, render_template, request, redirect, url_for
from pymongo import MongoClient
import re

app = Flask(__name__)

# MongoDB 連接（根據你的本地 MongoDB 配置進行調整）
client = MongoClient('mongodb://localhost:27017/')
db = client['local']  # 資料庫名稱
collection = db['startup_log']  # 集合名稱

@app.route('/')
def index():
    # 獲取搜尋關鍵字
    search_query = request.args.get('search', '')

    # 構建搜尋條件，使用 $or 查找關鍵字出現在任意欄位的文檔
    if search_query:
        search_condition = {
            "$or": [
                {"name": {"$regex": search_query, "$options": "i"}},
                {"description": {"$regex": search_query, "$options": "i"}},
                {"profession": {"$regex": search_query, "$options": "i"}}
            ]
        }
        data = list(collection.find(search_condition))
    else:
        # 如果沒有搜尋關鍵字，則顯示所有文檔
        data = list(collection.find({}))

    # 將 ObjectId 轉換為字符串，以便在 HTML 中顯示
    for doc in data:
        doc['_id'] = str(doc['_id'])
    
    return render_template('123-2.html', data=data, search_query=search_query)

@app.route('/create')
def create():
    # 顯示新增資料的表單頁面
    return render_template('search1.html')

@app.route('/create', methods=['GET', 'POST'])
def add_data():
    if request.method == 'POST':
        # 從表單中獲取基本數據
        name = request.form.get('name')
        description = request.form.get('description')
        profession = request.form.get('profession')

        # 基本字段
        new_doc = {
            'name': name,
            'description': description,
            'profession': profession
        }

        # 獲取動態新增的欄位名稱和屬性
        field_names = request.form.getlist('fieldName[]')
        field_values = request.form.getlist('fieldValue[]')

        # 將動態欄位添加到文檔中
        for i in range(len(field_names)):
            field_name = field_names[i]
            field_value = field_values[i]
            new_doc[field_name] = field_value

        # 將新數據插入到 MongoDB 集合中
        collection.insert_one(new_doc)

        # 提交後重定向到首頁以顯示新添加的數據
        return redirect(url_for('index'))

    # GET 請求時顯示新增資料的表單頁面
    return render_template('search1.html')



if __name__ == '__main__':
    app.run(host="0.0.0.0", port=8080, debug=True, use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8080
 * Running on http://172.18.34.158:8080
Press CTRL+C to quit
127.0.0.1 - - [04/Nov/2024 10:39:09] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/Nov/2024 10:39:09] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [04/Nov/2024 10:39:27] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/Nov/2024 10:39:27] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [04/Nov/2024 10:39:35] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/Nov/2024 10:40:37] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/Nov/2024 10:40:37] "GET /favicon.ico HTTP/1.1" 404 -
